<a href="https://colab.research.google.com/github/FIGARO79/GanaBaloto/blob/main/GanaBaloto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

if not os.path.exists('/content/drive/MyDrive/GanaBaloto'):
  print("Installing and Running For First Time.")
  %cd /content/drive/MyDrive
  !git clone https://github.com/FIGARO79/GanaBaloto.git
  !cp -r /content/drive/MyDrive/GanaBaloto
else:
  print("Repository already exist. Continue...")
  %cd /content/drive/MyDrive/GanaBaloto

Mounted at /content/drive
Installing and Running For First Time.
/content/drive/MyDrive
Cloning into 'GanaBaloto'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 38 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 117.78 KiB | 6.54 MiB/s, done.
Resolving deltas: 100% (13/13), done.
cp: missing destination file operand after '/content/drive/MyDrive/GanaBaloto'
Try 'cp --help' for more information.


# Ejecutar GanaBaloto

In [ ]:


import pandas as pd
from IPython.display import display, HTML
import random
from itertools import combinations
import numpy as np


# Update the file path to include the full path from your Google Drive
file_path = '/content/drive/MyDrive/GanaBaloto/resultados_aciertos.xlsx'
sheet_data = pd.read_excel(file_path, sheet_name='data')
sheet_data = sheet_data.dropna(subset=['SB'])
sheet_data['SB'] = sheet_data['SB'].astype(int)


#Calculate duplicate draws

combinations_df = sheet_data[['B1', 'B2', 'B3', 'B4', 'B5', 'SB']]
duplicate_mask = combinations_df.duplicated(keep=False)
duplicate_combinations = sheet_data[duplicate_mask]

if duplicate_combinations.empty:
    print("No hay combinaciones duplicadas.")
else:
    print("Duplicate Combinations:")
    display(duplicate_combinations)

# Calculate frequencies for all draws
all_draws_frequencies = {}
columns_to_analyze = ['B1', 'B2', 'B3', 'B4', 'B5']
for column in columns_to_analyze:
    all_draws_frequencies[column] = sheet_data[column].value_counts().head(5).index.tolist()
all_draws_frequencies["Super Balota"] = sheet_data["SB"].value_counts().head(5).index.tolist()

# Filter data for draws with 'Premios 5+1' > 0
filtered_data = sheet_data[sheet_data['Premios 5+1'] > 0]

# Calculate frequencies for filtered draws
refined_predictions = {}
for column in columns_to_analyze:
    refined_predictions[column] = filtered_data[column].value_counts().head(5).index.tolist()
refined_predictions["Super Balota"] = filtered_data['SB'].value_counts().head(5).index.tolist()

df_all_draws = pd.DataFrame(all_draws_frequencies).T.rename(columns={i: f'Top {i+1}' for i in range(5)})
df_refined_predictions = pd.DataFrame(refined_predictions).T.rename(columns={i: f'Top {i+1}' for i in range(5)})


# Combination Generation

def calculate_probability_numpy(combination, sb, frequencies_df):
    """Calculates the frequency score for a given combination using NumPy."""
    total_draws = frequencies_df.shape[0]
    probability = 0

    for i, number in enumerate(combination):
        column_name = f'B{i+1}'
        # Use NumPy to calculate frequency and probability
        column_data = frequencies_df[column_name].to_numpy()
        frequency = np.sum(column_data == number)
        probability += frequency / total_draws

    # Use NumPy for Super Balota calculation
    sb_data = frequencies_df['SB'].to_numpy()
    sb_frequency = np.sum(sb_data == sb)
    probability += sb_frequency / total_draws

    return probability # Changed this line to return the calculated probability

def generate_probable_combinations(predictions, num_combinations=10, frequencies_df=sheet_data):  # Add frequencies_df as argument
    """
    Generates combinations based on the most frequent numbers and calculates their probabilities.

    Args:
        predictions (dict): Dictionary containing the most frequent numbers for each column.
        num_combinations (int): Number of combinations to generate.
        frequencies_df (pd.DataFrame): DataFrame containing the frequency data.

    Returns:
        list: A list of probable combinations with their probabilities.
    """
    all_probable_numbers = [number for sublist in predictions.values() for number in sublist]
    # Ensure unique numbers while preserving order (using dict.fromkeys)
    unique_probable_numbers = list(dict.fromkeys(all_probable_numbers))

    probable_combinations = []
    for _ in range(num_combinations):
        combination = random.sample(unique_probable_numbers, 5)  # Pick 5 unique numbers
        sb = random.choice(predictions["Super Balota"])  # Pick Super Balota from its predictions

        probability = calculate_probability_numpy(combination, sb, frequencies_df) # Calculate probability
        probable_combinations.append((combination, sb, probability))  # Store with probability

    return probable_combinations


def jugada_manual(frequencies_df=None):
    while True:
        try:
            combination = []
            for i in range(5):
                while True:
                    number = int(input(f"Ingrese el número {i + 1} de la combinación (1-43): \n"))
                    if 1 <= number <= 43 and number not in combination:
                        combination.append(number)
                        break
                    else:
                        print("Número inválido o repetido. Ingrese un número único entre 1 y 43.")

            while True:
                sb = int(input("Ingrese el número de la Super Balota (1-16): \n"))
                if 1 <= sb <= 16:
                    break
                else:
                    print("Número inválido. Ingrese un número entre 1 y 16.")

            probability = calculate_probability_numpy(combination, sb, frequencies_df)
            print(f"\nCombinación: {combination}, Super Balota: {sb}")
            print(f"Probabilidad de acierto: {probability:.2%}")

            return combination, sb, probability

        except ValueError:
            print("Entrada inválida. Por favor, ingrese números enteros.")

# Generate and print 10 probable combinations with probabilities
probable_combinations = generate_probable_combinations(refined_predictions, num_combinations=10)  # Pass sheet_data

# Create a list to store the data for the DataFrame
data = []
for combination, sb, probability in probable_combinations:
    data.append([', '.join(map(str, combination)), sb, probability])

df_combinations = pd.DataFrame(data, columns=['Combination', 'Super Balota', 'Probability'], index=range(1, 11))
df_combinations['Probability'] = df_combinations['Probability'].map('{:.2%}'.format)

#print(f"\nCombinaciones con Probabilidadesde acierto:")
#print(df_combinations)
#display(HTML(df_combinations.to_html(escape=False)))

# Display tables side-by-side using HTML
# The following line was modified to use display(HTML())
html_str = f"""
<table><tr>
<td>
<h3>Números más frecuentes (todos los sorteos)</h3>
{df_all_draws.to_html()}
</td>
<td>
<h3>Números más frecuentes (solo sorteos con premio 5+1)</h3>
{df_refined_predictions.to_html()}
</td>
</tr></table>
"""
display(HTML(html_str))

#########

print("\nDesea ingresar una jugada manual? s/n \n")
response = input()
while response.lower() == "s":
    combination, sb, probability = jugada_manual(frequencies_df=sheet_data)  # Pass sheet_data here

    # Ask if the user wants to enter another manual combination
    response = input("\nDesea ingresar otra jugada manual? s/n: \n")  # Corrected indentation

# Proceed to automatic combinations if the user enters 'n'
if response.lower() == "n":
    print("\nSe generan jugadas automaticas.\n")
else:
    print("\nRespuesta no válida. Procediendo a jugadas automáticas.") # Handle unexpected input

##########

html_str = f"""
<table><tr>
<td>
<h3>Combinaciones automáticas con Probabilidades de acierto</h3>
{df_combinations.to_html()}
</td>
</tr></table>
"""
display(HTML(html_str))


print("\nDesea generar más combinaciones: s/n\n")
response = input()

while True:
  if response == "s":  # Corrección: usar == para comparación
    # Volver a ejecutar la función con los mismos parámetros
      probable_combinations = generate_probable_combinations(refined_predictions, num_combinations=10, frequencies_df=sheet_data)

      # Actualizar el DataFrame con las nuevas combinaciones
      data = []
      for combination, sb, probability in probable_combinations:
          data.append([', '.join(map(str, combination)), sb, probability])

      df_combinations = pd.DataFrame(data, columns=['Combination', 'Super Balota', 'Probability'], index=range(1, 11))
      df_combinations['Probability'] = df_combinations['Probability'].map('{:.2%}'.format)

      # Mostrar el DataFrame actualizado
      display(HTML(df_combinations.to_html()))

      print("\nDesea generar más combinaciones: s/n")
      response = input() #added to continue or exit after generating more combinations
  elif response == "n":
      print("\nMucha suerte con tus números.")
      break #Exit the while loop
  else:
      print("\nRespuesta no válida. Por favor, ingrese 's' o 'n'.")
      response = input()

No hay combinaciones duplicadas.


,Top 1,Top 2,Top 3,Top 4,Top 5
B1,1,2,3,6,5
B2,14,8,20,11,12
B3,25,20,22,24,28
B4,34,37,35,36,30
B5,43,42,41,40,39
Super Balota,11,7,13,16,3
,Top 1,Top 2,Top 3,Top 4,Top 5
B1,8,5,9,2,6
B2,6,9,15,16,11
B3,19,13,28,14,34



Desea ingresar una jugada manual? s/n 

n

Se generan jugadas automaticas.



,Combination,Super Balota,Probability
1,"13, 31, 2, 41, 16",14,10.50%
2,"11, 18, 7, 14, 16",1,14.62%
3,"7, 34, 5, 31, 18",7,18.88%
4,"5, 19, 41, 18, 14",15,18.25%
5,"27, 16, 28, 23, 6",15,16.12%
6,"35, 26, 7, 18, 38",7,16.75%
7,"31, 19, 15, 9, 13",1,14.62%
8,"16, 23, 8, 18, 28",13,16.75%
9,"27, 14, 23, 43, 13",15,14.50%
10,"23, 34, 6, 18, 41",14,19.00%



Desea generar más combinaciones: s/n

n

Mucha suerte con tus números.
